In [1]:
import torch
import pandas as pd
import re
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm

c:\Users\ashis\Downloads\DataScience Training\DATA_SCIENCE_TRAINING\5- DEEP LEARNING\54- TRANSFORMER\sentenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# Load a pretrained BERT model for sentiment analysis
MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"  # This model predicts sentiment on a 5-point scale
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)  # Load tokenizer
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)  # Load model
model.to(device)  # Move model to the selected device (GPU/CPU)
model.eval()  # Set model to evaluation mode (important for inference)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [4]:
# Load dataset
df = pd.read_csv('IMDB Dataset.csv')

In [5]:
pd.set_option('display.max_colwidth',None)
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [6]:
text = "Unbelievably great experience!"
tokens = tokenizer.tokenize(text)
print(tokens)

['un', '##beli', '##eva', '##bly', 'great', 'experience', '!']


In [7]:
# ======= Preprocessing Steps =======

# 1️⃣ Remove rows where 'review' is NaN or empty
df = df.dropna(subset=["review"])  # Drop rows where review is NaN
df = df[df["review"].str.strip() != ""]  # Remove empty strings after stripping whitespace

# 2️⃣ Define a function to clean the review
def clean_review(review):
    review = review.lower()  # Convert review to lowercase (helps for consistency)
    review = re.sub(r"https?://\S+|www\.\S+", "", review)  # Remove URLs (they don't contribute to sentiment)
    review = re.sub(r"<.*?>", "", review)  # Remove HTML tags (useful if data is scraped from the web)
    review = re.sub(r"[^\x00-\x7F]+", "", review)  # Remove non-ASCII characters (avoids encoding issues)
    review = re.sub(r"\s+", " ", review)  # Replace multiple spaces with a single space (cleans up formatting)
    review = re.sub(r"[^a-zA-Z0-9\s.,!?']", "", review)  # Keep only letters, numbers, and basic punctuation
    return review.strip()  # Remove leading/trailing spaces

# Apply review cleaning to the dataset
df["review"] = df["review"].apply(clean_review)

# 3️⃣ Truncate long reviews to avoid exceeding BERT's max length (512 tokens)
df["review"] = df["review"].apply(lambda x: " ".join(x.split()[:250]))  # Keeps first 250 words (reasonable limit)

In [8]:
df1=df.head(50)

In [9]:
df1.head()

,review,sentiment
0,"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. forget pretty pictures painted for mainstream audiences, forget charm, forget romance...oz doesn't mess around. the first episode i ever saw struck me as so nasty it was surreal, i couldn't say i was ready for it, but as i watched more, i developed a taste for oz, and got accustomed to the high levels of graphic violence. not just violence, but injustice crooked guards who'll be sold out for a nickel,",positive
1,"a wonderful little production. the filming technique is very unassuming very oldtimebbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master's of comedy and his life. the realism really comes home with the little things the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell's murals decorating every surface are terribly well done.",positive
2,"i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a lighthearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable even the well bread suspected serial killer. while some may be disappointed when they realize this is not match point 2 risk addiction, i thought it was proof that woody allen is still fully in control of the style many of us have grown to love.this was the most i'd laughed at one of woody's comedies in years dare i say a decade?. while i've never been impressed with scarlet johanson, in this she managed to tone down her sexy image and jumped right into a average, but spirited young woman.this may not be the crown jewel of his career, but it was wittier than devil wears prada and more interesting than superman a great comedy to go see with friends.",positive
3,"basically there's a family where a little boy jake thinks there's a zombie in his closet his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents descent dialogs. as for the shots wi

In [10]:
#maximum 250 words as per our setup
len(df1.review[0].split())

250

In [11]:
# ======= Sentiment Prediction Function =======
def predict_sentiment(text_list):
    predictions = []  # List to store predictions
    
    for text in tqdm(text_list, desc="Processing"):  # Use tqdm for progress tracking
        # Tokenize text (convert words into numerical format that BERT understands)
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Move tensors to the selected device (GPU/CPU)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Perform inference (predict sentiment)
        with torch.no_grad():  # Disable gradient calculation (saves memory and speeds up inference)
            outputs = model(input_ids, attention_mask=attention_mask)  # Forward pass
            logits = outputs.logits  # Get raw model outputs
            probs = F.softmax(logits, dim=1)  # Convert logits to probabilities
            sentiment = torch.argmax(probs, dim=1).item()  # Get the index of the highest probability
            print("probs : ",probs)
            print("sentiment : ",sentiment)
            print("************")

            # Map numerical sentiment labels to human-readable labels
            sentiment_labels = {
                0: "Very Negative",
                1: "Negative",
                2: "Neutral",
                3: "Positive",
                4: "Very Positive"
            }
            
            predictions.append(sentiment_labels[sentiment])  # Append prediction to list
    
    return predictions  # Return the list of predicted sentiments

# Apply sentiment prediction function to dataset
df1["predicted_sentiment"] = predict_sentiment(df1["review"].tolist())


Processing:   2%|▏         | 1/50 [00:01<01:27,  1.79s/it]

probs :  tensor([[0.0921, 0.2058, 0.2203, 0.3162, 0.1655]])
sentiment :  3
************


Processing:   4%|▍         | 2/50 [00:02<00:44,  1.08it/s]

probs :  tensor([[0.0011, 0.0032, 0.0235, 0.4054, 0.5668]])
sentiment :  4
************


Processing:   6%|▌         | 3/50 [00:02<00:32,  1.47it/s]

...

Processing:  98%|█████████▊| 49/50 [00:15<00:00,  3.25it/s]

probs :  tensor([[0.1137, 0.3699, 0.2817, 0.1865, 0.0483]])
sentiment :  1
************


Processing: 100%|██████████| 50/50 [00:15<00:00,  3.19it/s]

probs :  tensor([[0.0098, 0.2169, 0.7526, 0.0197, 0.0010]])
sentiment :  2
************



C:\Users\ashis\AppData\Local\Temp\ipykernel_15192\4118152146.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["predicted_sentiment"] = predict_sentiment(df1["review"].tolist())


In [12]:
df1

,review,sentiment,predicted_sentiment
0,"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. forget pretty pictures painted for mainstream audiences, forget charm, forget romance...oz doesn't mess around. the first episode i ever saw struck me as so nasty it was surreal, i couldn't say i was ready for it, but as i watched more, i developed a taste for oz, and got accustomed to the high levels of graphic violence. not just violence, but injustice crooked guards who'll be sold out for a nickel,",positive,Positive
1,"a wonderful little production. the filming technique is very unassuming very oldtimebbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master's of comedy and his life. the realism really comes home with the little things the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell's murals decorating every surface are terribly well done.",positive,Very Positive
2,"i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a lighthearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable even the well bread suspected serial killer. while some may be disappointed when they realize this is not match point 2 risk addiction, i thought it was proof that woody allen is still fully in control of the style many of us have grown to love.this was the most i'd laughed at one of woody's comedies in years dare i say a decade?. while i've never been impressed with scarlet johanson, in this she managed to tone down her sexy image and jumped right into a average, but spirited young woman.this may not be the crown jewel of his career, but it was wittier than devil wears prada and more interesting than superman a great comedy to go see with friends.",positive,Positive
3,"basically there's a family where a little boy jake thinks there's a zombie in his closet his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well 